# **HUNGARIAN ALGORITHM**
***
The **Hungarian Algorithm** (also known as the Munkres-Kuhn Algorithm) is a combinatorial optimization algorithm for solving the **assignment problem**. The assignment problem involves finding the most optimal way to pair items from two sets (e.g., workers to tasks) such that the total cost is minimized (or profit maximized) while ensuring a one-to-one assignment.

The algorithm works on a cost matrix, where each entry represents the cost of assigning a specific task to a specific worker.

The assignment problem is equivalent to finding a maximum-weight matching in a weighted bipartite graph. In the context of a bipartite graph:
* One set of nodes represents workers.
* The other set represents tasks.
* Edges between nodes represent possible assignments, with weights as the cost (or profit).

In a weighted bipartite graph, an equality graph is a subgraph where the edges satisfy a specific equality condition derived from a pair of labeling functions on the vertex sets. 
\(l(u)+l(v)=w(u,v)\)
*** 
- Input: A **cost matrix** \( C \) of size \( nxn \), where \( C[u][v] \) is the cost of assigning node \( u \) in \( U \) to node \( v \) in \( V \). 
- Output: A **perfect matching** \( M \) (set of pairs \((u, v)\)) with minimum total cost.



In [2]:
import numpy as np

class HungarianAlgorithmGraph:
    def __init__(self, cost_matrix):
        """
        Initialize the HungarianAlgorithmGraph with a cost matrix.
        param cost_matrix: A square matrix where cost_matrix[u][v] represents the cost of assigning u to v.
        """
        self.cost_matrix = cost_matrix  # The cost matrix for the assignment problem
        self.n = len(cost_matrix)  # Number of nodes in each set (U and V)
        self.U = set(range(self.n))  # Set of nodes in U
        self.V = set(range(self.n))  # Set of nodes in V
        self.matching = {}  # Dictionary to store the current matching
        self.labels_u = [0] * self.n  # Labels for nodes in U
        self.labels_v = [0] * self.n  # Labels for nodes in V
        self.slack = [float('inf')] * self.n  # Slack values for nodes in V

    def initialize_labels(self):
        """
        Initialize labels for U and V.
        Labels for U are set to the maximum cost in each row of the cost matrix,
        and labels for V are initialized to 0.
        """
        for u in range(self.n):
            self.labels_u[u] = max(self.cost_matrix[u])  # Max weight in row u for label_u[u]

    def find_augmenting_path(self, u, visited_u, visited_v, parent):
        """
        Attempt to find an augmenting path starting from node u in U.
        -param u: Current node in U
        -param visited_u: List tracking visited nodes in U
        -param visited_v: List tracking visited nodes in V
        -param parent: Parent array to reconstruct paths        """
        visited_u[u] = True  # Mark the current node in U as visited
        for v in range(self.n):  # Iterate over nodes in V
            if visited_v[v]:  # Skip if node v is already visited
                continue
            # Compute the reduced cost for edge (u, v)
            delta = self.labels_u[u] + self.labels_v[v] - self.cost_matrix[u][v]
            if delta == 0:  # If edge (u, v) is in the equality graph
                visited_v[v] = True  # Mark v as visited
                # If v is unmatched or we can find an augmenting path from its match
                if v not in self.matching or self.find_augmenting_path(self.matching[v], visited_u, visited_v, parent):
                    self.matching[v] = u  # Match u with v
                    return True
            else:
                # Update the slack for node v
                self.slack[v] = min(self.slack[v], delta)
        return False  # No augmenting path found

    def update_labels(self, visited_u, visited_v):
        """
        Update labels for U and V to reduce slack and maintain feasibility.
        -param visited_u: List tracking visited nodes in U
        -param visited_v: List tracking visited nodes in V
        """
        # Compute the smallest slack value
        delta = min(self.slack[v] for v in range(self.n) if not visited_v[v])
        # Update labels for visited nodes in U and V
        for u in range(self.n):
            if visited_u[u]:
                self.labels_u[u] -= delta
        for v in range(self.n):
            if visited_v[v]:
                self.labels_v[v] += delta
            else:
                self.slack[v] -= delta

    def augment(self):
        """
        Perform augmentation for each node in U to construct the optimal matching.
        """
        for u in range(self.n):  # Iterate over all nodes in U
            visited_u = [False] * self.n  # Reset visited for U
            visited_v = [False] * self.n  # Reset visited for V
            self.slack = [float('inf')] * self.n  # Reset slack values
            # Repeat until an augmenting path is found for u
            while not self.find_augmenting_path(u, visited_u, visited_v, {}):
                self.update_labels(visited_u, visited_v)  # Update labels if no path is found
                visited_u = [False] * self.n  # Reset visited for U
                visited_v = [False] * self.n  # Reset visited for V

    def solve(self):
        """
        Solve the assignment problem using the Hungarian algorithm.
        return: A list of optimal assignments and the total cost.
        """
        self.initialize_labels()  # Step 1: Initialize labels
        self.augment()  # Step 2: Find optimal matching through augmentation
        # Extract the resulting matching
        result = [(self.matching[v], v) for v in self.matching]
        # Compute the total cost of the matching
        optimal_cost = sum(self.cost_matrix[u][v] for u, v in result)
        return result, optimal_cost

# **Example of usage**

In [3]:
cost_matrix = np.array([
    [4, 2, 8],  # Costs for assignments of u_0
    [2, 3, 7],  # Costs for assignments of u_1
    [3, 1, 6]   # Costs for assignments of u_2
])

# Instantiate the HungarianAlgorithmGraph class
hungarian = HungarianAlgorithmGraph(cost_matrix)
assignment, cost = hungarian.solve()

# Output the results
print("Optimal Assignment:", assignment)  # Matching pairs (u, v)
print("Optimal Cost:", cost)  # Total cost of the optimal matching


Optimal Assignment: [(0, 2), (1, 1), (2, 0)]
Optimal Cost: 14


In [4]:
cost_matrix = np.random.randint(0, 101, (200, 200))
hungarian = HungarianAlgorithmGraph(cost_matrix)
matching, optimal_cost = hungarian.solve()
# Displaying the size of the matching and optimal cost (since output might be large)
len(matching), optimal_cost

(200, 19932)

# Hungarian Algorithm Example

We solve the minimization assignment problem using the Hungarian Algorithm for the following cost matrix \( C \):


\begin{bmatrix}
4 & 2 & 8 \\
2 & 3 & 7 \\
3 & 1 & 6
\end{bmatrix}


---

### Step 1: Initialization
Each label for 𝑈 is the maximum value in the corresponding row of the cost matrix.

   - label_U[0] = max(4, 2, 8) = 8, 
   - label_U[1] = max(2, 3, 7) = 7, 
   - label_U[2] = max(3, 1, 6) = 6

   Therefore:
   - label_U = [8, 7, 6]

Initially, all labels for 𝑉 are set to 0:

   - label_𝑉=[0,0,0]


Resulting matrix:
\begin{bmatrix}
2 & 0 & 6 \\
0 & 1 & 5 \\
2 & 0 & 5
\end{bmatrix}

---

### Step 2: Equality Graph
The equality graph is constructed by including edges (𝑢,𝑣) such that:

\begin{equation}
label_𝑈[𝑢]+label_𝑉[𝑣]=𝐶[𝑢][𝑣]
\end{equation}
​
​Verifying edges:
- For \( (u = 0, v = 0): 8 + 0 != 4 \), no edge.
- For \( (u = 0, v = 1): 8 + 0 != 2 \), no edge.
- For \( (u = 0, v = 2): 8 + 0 = 8 \), include \( (0, 2) \).  
- For \( (u = 1, v = 0): 7 + 0 != 2 \), no edge.
- For \( (u = 1, v = 1): 7 + 0 != 3 \), no edge.
- For \( (u = 1, v = 2): 7 + 0 = 7 \), include \( (1, 2) \).  
- For \( (u = 2, v = 0): 6 + 0 != 3 \), no edge.
- For \( (u = 2, v = 1): 6 + 0 != 1 \), no edge.
- For \( (u = 2, v = 2): 6 + 0 = 6 \), include \( (2, 2) \). 

**Equality Graph:** \( \{(0, 2), (1, 2), (2, 2)\} \).


---

## **Step 3: Finding an Augmenting Path**

The goal in this step is to find a path in the equality graph (constructed in Step 2) that allows us to "augment" the matching by either:

- Assigning an unmatched 𝑢 to an unmatched 𝑣, or
- Rearranging existing matches to create a valid matching that includes the current 𝑢.

We process each 𝑢 (rows in the cost matrix) one by one.
1. Start with  u = 0:
   - u = 0 is connected to v = 2 (because 
\begin{equation}
   label_𝑈[0]+label_𝑉[2]=𝐶[0][2]
\end{equation}

   - Since v = 2 is unmatched, match (u = 0, v = 2).

   **Matching:** \( \{(0, 2)\} \).

2. Move to  u = 1 :
   - u = 1  is connected to  v = 2 , but  v = 2  is already matched with  u = 0 .
   - Look for an augmenting path through  v = 2 . No augmenting path is found.

---
## **Step 4: Updating Labels**

Since no perfect matching exists yet, update the labels to include more edges in the equality graph.

\begin{equation}
slack[𝑣]=min⁡(label𝑈[𝑢]+label𝑉[𝑣]−𝐶[𝑢][𝑣]))
\end{equation}

Calculate slack for v=0,1,2:

- slack[0]=min(8+0−4,7+0−2,6+0−3)=min(4,5,3)=3.

- slack[1]=min(8+0−2,7+0−3,6+0−1)=min(6,4,5)=4.

- slack[2]=min(8+0−8,7+0−7,6+0−6)=min(0,0,0)=0.

---
## **Step 5: Find a new augmentating path**

With the updated labels, re-check the equality graph.

New edges in the equality graph:
For:

- u=0,v=0:5+0=4, no edge.

- u=0,v=1:5+0=2, no edge.

- u=0,v=2:5+3=8, include (0,2).

- u=1,v=0:4+0=2, no edge.

- u=1,v=1:4+0=3, include (1,1).

- u=1,v=2:4+3=7, include (1,2).

u=2,v=0:3+0=3, include (2,0).
Updated Equality Graph: {(0,2),(1,1),(2,0)}.
Find a matching:
- u=0: Match (0,2).

- u=1: Match (1,1).

- u=2: Match (2,0).
Final Matching: {(0,2),(1,1),(2,0)}.

---
## **Final Step:  Compute Total Cost**
The total cost of the matching is:
\begin{equation}
𝐶[0][2]+𝐶[1][1]+𝐶[2][0]=8+3+3=14.
\end{equation}

This completes the Hungarian Algorithm for the given cost matrix.
